In [1]:
import polars as pl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import glob


In [27]:
player_plays = pl.read_csv(
    "data/player_play.csv",
    # columns=[
    #     "gameId", "playId", "nflId", "teamAbbr", "hadRushAttempt", "rushingYards",
    #     "hadDropback", "passingYards", "sackYardsAsOffense", "hadPassReception",
    #     "receivingYards", "wasTargettedReceiver", "yardageGainedAfterTheCatch", 
    #     "forcedFumbleAsDefense", "quarterbackHit", "sackYardsAsDefense", "soloTackle", 
    #     "tackleForALoss", "tackleForALossYardage", "hadInterception", "penaltyYards", 
    #     "wasInitialPassRusher", "causedPressure", "inMotionAtBallSnap", "shiftSinceLineset", 
    #     "motionSinceLineset", "wasRunningRoute", "routeRan", "pressureAllowedAsBlocker",
    #     "pff_defensiveCoverageAssignment", "pff_primaryDefensiveCoverageMatchupNflId"
    # ]
)

Why is route-running effeiency important? Easy – take a look at timeToPressureAsPassRusher & tackleForALoss. What is the average time a QB has to get a pass off before he's sacked. On 

In [42]:
tfl = player_plays.filter(
    (pl.col("tackleForALoss") == True)
)
tfl

gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,receivingYards,wasTargettedReceiver,yardageGainedAfterTheCatch,fumbles,fumbleLost,fumbleOutOfBounds,assistedTackle,forcedFumbleAsDefense,halfSackYardsAsDefense,passDefensed,quarterbackHit,sackYardsAsDefense,safetyAsDefense,soloTackle,tackleAssist,tackleForALoss,tackleForALossYardage,hadInterception,interceptionYards,fumbleRecoveries,fumbleRecoveryYards,penaltyYards,penaltyNames,wasInitialPassRusher,causedPressure,timeToPressureAsPassRusher,getOffTimeAsPassRusher,inMotionAtBallSnap,shiftSinceLineset,motionSinceLineset,wasRunningRoute,routeRan,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_defensiveCoverageAssignment,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId
i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2022090800,467,37075,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-8,0,1,0,1,8,0,0,0,0,0,"""NA""","""1""",true,"""2.5""","""0.935999989509583""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,550,41239,"""LA""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-1,0,1,0,1,1,0,0,0,0,0,"""NA""","""1""",true,"""2.29999995231628""","""0.845000028610229""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,1315,42395,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-10,0,1,0,1,10,0,0,0,0,0,"""NA""","""1""",true,"""2.20000004768372""","""0.994000017642975""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,1504,38577,"""LA""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-4,0,1,0,1,4,0,0,0,0,0,"""NA""","""1""",true,"""2""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,1836,37075,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,"""NA""","""0""",false,"""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022110700,2465,37145,"""BAL""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-2,0,1,0,1,2,0,0,0,0,0,"""NA""","""1""",true,"""3.59999990463257""","""0.714999973773956""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022110700,2637,44828,"""BAL""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-9,0,1,0,1,9,0,0,0,0,0,"""NA""","""1""",true,"""1.60000002384186""","""0.799000024795532""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022110700,3037,53457,"""NO""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,4,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""0.944999992847443""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""


In [30]:
# Filter for plays that are relevant to quarterback scrambling (e.g., dropbacks with defensive pressures)
scramble_plays = player_plays.filter(
    (pl.col("hadDropback") == True)
)

In [38]:
inMotionAtBallSnapDf = player_plays.filter(
    (pl.col("inMotionAtBallSnap") == "TRUE")
)

In [39]:
inMotionAtBallSnapDf

gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,receivingYards,wasTargettedReceiver,yardageGainedAfterTheCatch,fumbles,fumbleLost,fumbleOutOfBounds,assistedTackle,forcedFumbleAsDefense,halfSackYardsAsDefense,passDefensed,quarterbackHit,sackYardsAsDefense,safetyAsDefense,soloTackle,tackleAssist,tackleForALoss,tackleForALossYardage,hadInterception,interceptionYards,fumbleRecoveries,fumbleRecoveryYards,penaltyYards,penaltyNames,wasInitialPassRusher,causedPressure,timeToPressureAsPassRusher,getOffTimeAsPassRusher,inMotionAtBallSnap,shiftSinceLineset,motionSinceLineset,wasRunningRoute,routeRan,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_defensiveCoverageAssignment,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId
i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2022090800,80,47857,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""TRUE""","""FALSE""","""TRUE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,101,53079,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""TRUE""","""FALSE""","""TRUE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,212,47879,"""BUF""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""TRUE""","""FALSE""","""TRUE""","""1""","""IN""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,236,52536,"""BUF""",0,0,0,0,0,1,26,1,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""TRUE""","""FALSE""","""TRUE""","""1""","""CORNER""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,299,44881,"""LA""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""TRUE""","""FALSE""","""TRUE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022110700,2286,54604,"""BAL""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""TRUE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022110700,2387,45244,"""NO""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""TRUE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022110700,2902,45268,"""BAL""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""TRUE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""


In [31]:
scramble_plays

gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,receivingYards,wasTargettedReceiver,yardageGainedAfterTheCatch,fumbles,fumbleLost,fumbleOutOfBounds,assistedTackle,forcedFumbleAsDefense,halfSackYardsAsDefense,passDefensed,quarterbackHit,sackYardsAsDefense,safetyAsDefense,soloTackle,tackleAssist,tackleForALoss,tackleForALossYardage,hadInterception,interceptionYards,fumbleRecoveries,fumbleRecoveryYards,penaltyYards,penaltyNames,wasInitialPassRusher,causedPressure,timeToPressureAsPassRusher,getOffTimeAsPassRusher,inMotionAtBallSnap,shiftSinceLineset,motionSinceLineset,wasRunningRoute,routeRan,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_defensiveCoverageAssignment,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId
i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,bool,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2022090800,56,46076,"""BUF""",0,0,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,122,46076,"""BUF""",0,0,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,167,46076,"""BUF""",0,0,1,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,212,46076,"""BUF""",0,0,1,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022090800,236,46076,"""BUF""",0,0,1,26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022110700,3299,37110,"""NO""",0,0,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""NA""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022110700,3323,37110,"""NO""",0,0,1,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""NA""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""
2022110700,3347,37110,"""NO""",0,0,1,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""NA""","""NA""",false,"""NA""","""NA""","""FALSE""","""FALSE""","""FALSE""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA"""


In [11]:
# Load and process tracking data for all weeks
tracking_files = glob.glob("data/tracking_week_*.csv")
tracking_data = []
for file in tracking_files:
    week_data = pl.read_csv(
        file,
        columns=[
            "gameId", "playId", "nflId", "displayName", "frameId", "frameType",
            "time", "jerseyNumber", "club", "playDirection", "x", "y", "s", "a",
            "dis", "o", "dir", "event"
        ],
        null_values= ["NA"]
    )
    # Feature engineering: Aggregate relevant tracking metrics per play
    week_data_grouped = (
        week_data.group_by(["gameId", "playId"])
        .agg([
            pl.col("s").mean().alias("avg_speed"),
            pl.col("a").mean().alias("avg_acceleration"),
            pl.col("o").mean().alias("avg_orientation"),
            pl.col("dir").mean().alias("avg_direction"),
            (pl.col("displayName") == "QB").sum().alias("qb_in_play")
        ])
    )
    tracking_data.append(week_data_grouped)

# Concatenate all tracking data for each week
all_tracking_data = pl.concat(tracking_data)

In [18]:
# Merge tracking data with the filtered scramble plays
model_data = scramble_plays.join(all_tracking_data, on=["gameId", "playId"])

# Select relevant features for training
features = [
    "avg_speed", "avg_acceleration", "avg_orientation", "avg_direction", "qb_in_play",
    "sackYardsAsOffense", "tackleForALoss", "penaltyYards", "pff_defensiveCoverageAssignment",
    "shiftSinceLineset", "motionSinceLineset"
]
target = "causedPressure"  # Adjust target as needed for defensive prediction

# Prepare the data for modeling
X = model_data.select(features)
y = model_data.select([target]).to_numpy().flatten()



In [19]:
X

avg_speed,avg_acceleration,avg_orientation,avg_direction,qb_in_play,sackYardsAsOffense,tackleForALoss,penaltyYards,pff_defensiveCoverageAssignment,shiftSinceLineset,motionSinceLineset
f64,f64,f64,f64,u32,i64,i64,i64,str,str,str


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model (Random Forest for simplicity)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Show feature importance to interpret the model (optional)
feature_importances = model.feature_importances_
for feature, importance in zip(features, feature_importances):
    print(f"{feature}: {importance:.4f}")